# Laboratorio 5 — NLP: clasificación de sentimiento en reseñas (TF-IDF + modelo lineal)

Construirás un clasificador de texto con un dataset pequeño embebido y un pipeline reproducible.

## Objetivos
- Preprocesar texto y vectorizar con TF-IDF.
- Entrenar un modelo lineal y evaluarlo.
- Inspeccionar palabras/patrones que empujan una predicción.

## Requisitos
- scikit-learn
- pandas


## 1) Dataset mínimo (embebido)

Para evitar dependencias externas, usamos un dataset pequeño embebido. 
En ejercicios se propone ampliarlo con más datos o un dataset público.


In [1]:
import pandas as pd

data = [
    ("El producto llegó tarde y la calidad es muy mala.", 0),
    ("Excelente, funciona perfecto y el envío fue rápido.", 1),
    ("No cumple lo prometido. Decepción total.", 0),
    ("Muy recomendable, buena relación calidad-precio.", 1),
    ("Se rompió al segundo día. No lo compraría otra vez.", 0),
    ("Atención al cliente impecable y gran rendimiento.", 1),
    ("Materiales baratos, acabado deficiente.", 0),
    ("Lo uso a diario y estoy encantado.", 1),
    ("No era compatible y además venía incompleto.", 0),
    ("Superó mis expectativas, repetiré compra.", 1),
    ("La batería dura poco y se calienta demasiado.", 0),
    ("Instalación sencilla y resultados excelentes.", 1),
]
df = pd.DataFrame(data, columns=["texto","sentimiento"])  # 1=positivo, 0=negativo
df


,texto,sentimiento
0,El producto llegó tarde y la calidad es muy mala.,0
1,"Excelente, funciona perfecto y el envío fue rá...",1
2,No cumple lo prometido. Decepción total.,0
3,"Muy recomendable, buena relación calidad-precio.",1
4,Se rompió al segundo día. No lo compraría otra...,0
5,Atención al cliente impecable y gran rendimiento.,1
6,"Materiales baratos, acabado deficiente.",0
7,Lo uso a diario y estoy encantado.,1
8,No era compatible y además venía incompleto.,0
9,"Superó mis expectativas, repetiré compra.",1


## 2) Split y pipeline TF-IDF + Linear SVM

Para texto, un baseline muy fuerte suele ser: `TfidfVectorizer` + `LinearSVC` (o LogisticRegression).


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df["texto"], df["sentimiento"], test_size=0.3, random_state=42, stratify=df["sentimiento"]
)

pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ("clf", LinearSVC())
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))


              precision    recall  f1-score   support

           0      1.000     0.500     0.667         2
           1      0.667     1.000     0.800         2

    accuracy                          0.750         4
   macro avg      0.833     0.750     0.733         4
weighted avg      0.833     0.750     0.733         4



## 3) Probar frases nuevas

Un modelo útil debe generalizar a frases no vistas. Probamos inferencia rápida.


In [3]:
samples = [
    "La calidad es excelente y el acabado muy bueno.",
    "Ha sido una compra terrible, no funciona y llegó roto.",
    "Cumple, pero por ese precio esperaba algo mejor."
]
pred = pipe.predict(samples)
proba_like = pred  # LinearSVC no da probas por defecto

for s, p in zip(samples, pred):
    print(f"[{'POS' if p==1 else 'NEG'}] {s}")


[NEG] La calidad es excelente y el acabado muy bueno.
[NEG] Ha sido una compra terrible, no funciona y llegó roto.
[POS] Cumple, pero por ese precio esperaba algo mejor.


## 4) Interpretabilidad: términos más influyentes

En modelos lineales, el peso de cada término da pistas. 
Cuidado: con dataset pequeño, estas conclusiones son inestables.


In [4]:
import numpy as np

tfidf = pipe.named_steps["tfidf"]
clf = pipe.named_steps["clf"]

feature_names = np.array(tfidf.get_feature_names_out())
coef = clf.coef_.ravel()

top_pos = feature_names[np.argsort(coef)[-10:]][::-1]
top_neg = feature_names[np.argsort(coef)[:10]]

print("Top positivo:", list(top_pos))
print("Top negativo:", list(top_neg))


Top positivo: ['uso diario', 'diario', 'uso', 'lo uso', 'diario estoy', 'encantado', 'estoy encantado', 'estoy', 'recomendable buena', 'relación']
Top negativo: ['no', 'la', 'se', 'cumple lo', 'cumple', 'decepción', 'decepción total', 'lo prometido', 'prometido decepción', 'prometido']


## Ejercicios

### Ejercicio 1: Aumentar el dataset
Añade al menos 40 frases nuevas (20 positivas, 20 negativas) basadas en reseñas realistas. Reentrena y evalúa con un split reproducible.

**Entregables**
- Nuevo dataframe
- Métricas
- Breve reflexión sobre mejoras

**Criterios de evaluación**
- Datos coherentes y balanceados
- Pipeline correcto
- Conclusiones razonables

### Ejercicio 2: Comparar modelos
Compara `LinearSVC` con `LogisticRegression` (con `class_weight` si procede). Reporta F1 y analiza errores.

**Entregables**
- Tabla comparativa
- 2 ejemplos de error y análisis

**Criterios de evaluación**
- Comparación justa
- Métricas adecuadas
- Análisis de errores concreto

### Ejercicio 3: Reto avanzado: pipeline de limpieza
Implementa una función de preprocesado (minúsculas, eliminación de signos, stopwords opcional) y evalúa si mejora.

**Entregables**
- Función de limpieza
- Resultados antes/después

**Criterios de evaluación**
- Preprocesado razonable
- No rompe el pipeline
- Comparación basada en evidencia
